In [2]:
!pip install pyspark

In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "EPLRDD")
sc.setLogLevel("ERROR")

In [5]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print("Uploaded file:", file_name)

Saving EPL.csv to EPL (1).csv
Uploaded file: EPL (1).csv


In [6]:
epl_rdd = sc.textFile("EPL.csv", 4)
header = epl_rdd.first()
data = epl_rdd.filter(lambda row: row != header)
rows = data.map(lambda line: line.split(","))

In [8]:
def safe_int_conversion(value):
    try:
        # Attempt to convert to float first to handle '1.0' cases
        return int(float(value))
    except ValueError:
        # Handle cases that cannot be converted to float or int
        return None

season_goals = rows.map(lambda x: (x[5], safe_int_conversion(x[2]) + safe_int_conversion(x[3])) if safe_int_conversion(x[2]) is not None and safe_int_conversion(x[3]) is not None else None)
season_goals = season_goals.filter(lambda x: x is not None) # Filter out rows with conversion errors

season_total = season_goals.reduceByKey(lambda a, b: a + b)
highest_goals_season = season_total.takeOrdered(1, key=lambda x: -x[1])
print("Highest goals season:", highest_goals_season)

Highest goals season: [('2011-2012', 1066)]


In [9]:
lowest_goals_season = season_total.takeOrdered(1, key=lambda x: x[1])
print("Lowest goals season:", lowest_goals_season)

Lowest goals season: [('2006-2007', 931)]


In [12]:
def safe_int_conversion(value):
    try:
        # Attempt to convert to float first to handle '1.0' cases
        return int(float(value))
    except ValueError:
        # Handle cases that cannot be converted to float or int
        return None

team_goals = rows.flatMap(lambda x: [
    (x[0], safe_int_conversion(x[2])),
    (x[1], safe_int_conversion(x[3]))
]).filter(lambda x: x[1] is not None) # Filter out rows with conversion errors

team_total_goals = team_goals.reduceByKey(lambda a, b: a + b)

team_seasons = rows.flatMap(lambda x: [
    (x[0], {x[5]}),
    (x[1], {x[5]})
]).reduceByKey(lambda a, b: a.union(b)) \
  .mapValues(lambda s: len(s))

team_avg_goals = team_total_goals.join(team_seasons) \
                                 .mapValues(lambda x: x[0] / x[1])

highest_avg_team = team_avg_goals.takeOrdered(1, key=lambda x: -x[1])
print("Team with highest avg goals/season:", highest_avg_team)

Team with highest avg goals/season: [('Manchester United', 72.25)]


In [14]:
manu_matches = rows.filter(lambda x: x[0] == "Manchester United" or x[1] == "Manchester United")
total_matches = manu_matches.count()
manu_wins = manu_matches.filter(
    lambda x: (x[0] == "Manchester United" and x[4] == "H") or
              (x[1] == "Manchester United" and x[4] == "A")
).count()
manu_losses = manu_matches.filter(
    lambda x: (x[0] == "Manchester United" and x[4] == "A") or
              (x[1] == "Manchester United" and x[4] == "H")
).count()
manu_draws = manu_matches.filter(lambda x: x[4] == "D").count()
p_win = manu_wins / total_matches
p_loss = manu_losses / total_matches
p_draw = manu_draws / total_matches
print("P(Win):", p_win, " P(Loss):", p_loss, " P(Draw):", p_draw)

P(Win): 0.6359649122807017  P(Loss): 0.16885964912280702  P(Draw): 0.19517543859649122
